# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

1. Сравните метрики hit_rate@k, precision@k. Какую метрику использовать предпочтительно и почему? Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

В задачах, где есть необходимость рекомендовать пользователю совершенно новый продукт и есть внешнее ограничение (например k песен в превью), наверное лучше использовать метрику hit_rate@k:

- Плейлист с новыми исполнителями для юзера
- Предложенная подборка новых фильмов / сериалов / книг

А в задачах, где есть ограничение по количеству вывода и прямой запрос пользователя - precision@k:

- Витрина на главной странице сайта (если там расположено k объектов)
- Поисковый запрос
- Рассылка


Наверное hit_rate, как и hit_rate@k, стоит использовать в задачах, где мы делаем для пользователя на какой то регулярной основе подборку нового контента, но нет внешнего ограничения в количестве: 

- Плейлист с новинками (без превью) (клиент - пользователь, товар - трек)
- Подборка новых ресторанов (клиент - пользователь, товар - ресторан)
- Подборка новостей (клиент - пользователь, товар - ссылка на новость)

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.

Логарифм используется для сглаживания функции, как и в случае с logloss. Аналогично можно использовать (наверное) экспоненту и квадрат. 

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).

??? нужно выделить те, которые не были упомянуты в файле с вебинара??? 
(не смогла найти по поиску - все что нашла было перечислено в ноутбуке - в том числе в разделе другое. или он не считался??) 

4. boughted = [1, 3, 5, 7, 9, 11], recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8

- не поняла где ошибка с AP@8  (поэтому результаты не совпадают)
- не до конца разобралась с NDCG@8, RR@8  (нашла несколько реализаций, но не до конца понимаю как они работают(
- ERR@8 совсем не дался 

In [1]:
import numpy as np
import math

In [2]:
bought = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [3]:
precision_at_k_manual = (0+0+1+1+1+1)/8
precision_at_k_manual

0.5

In [4]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [5]:
precision_at_k(recommended, bought, k=8)

[False False  True  True  True  True]


0.5

In [6]:
recall_at_k_manual = (0+0+1+1+1+1)/6
recall_at_k_manual

0.6666666666666666

In [7]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [8]:
recall_at_k(recommended, bought, k=8)

0.6666666666666666

In [9]:
ap_k_manual = (0+0+1/3+2/4+3/5+4/6)/8
ap_k_manual 

0.26249999999999996

In [10]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    
    # Если i идет от единицы - разве функция будет захватывать первый элемент списка?
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

In [11]:
ap_k(recommended, bought, k=8)

[False False False False False False]
[False False  True False False False]
[False False  True  True False False]
[False False  True  True False  True]


0.16

In [12]:
#Нашла реализацию на Git'е. Пока не совсем понимаю как это уложить в формулу

def ndcg_p(ordered_data, p):
    if sum(ordered_data)==0:
        return 0
    else:
        indexloop = range(0, p)
        DCG_p = 0
        for index in indexloop:
            current_ratio=(2**(ordered_data[index])-1)*(math.log((float(index)+2), 2)**(-1))
            DCG_p = DCG_p + current_ratio
        np.sort(ordered_data)[::-1]
        K = len(ordered_data)
        indexloop = range(0, K)
        iDCG_p = 0
        for index in indexloop:
            current_ratio=(2**(ordered_data[index])-1)*((math.log((index+2), 2))**(-1))
            iDCG_p = iDCG_p + current_ratio
        return(DCG_p/iDCG_p)

In [13]:
ordered_data = np.isin(recommended, bought)[:8]
ndcg_p(ordered_data, 8)

1.0

In [14]:
def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs]) 

In [15]:
mean_reciprocal_rank(ordered_data)

<ipython-input-14-c51bd1fd0203>:2: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  rs = (np.asarray(r).nonzero()[0] for r in rs)


0.5

----

---